In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating environment at `C:\Users\email\OneDrive\Dokumente\Uni\Köln\21WS\ATIS3\GroupProject\ATIES3_group_project-main\Project.toml`


In [2]:
using JuMP, Plots, CPLEX, DataFrames, XLSX, Statistics#, IterTools, StatsPlots

# Read in Data

In [3]:
data = Dict()
data["time_series"] = DataFrame(XLSX.readtable("data_ATIS3_new.xlsx", "time_series")...)
data["p_scenarios"] = DataFrame(XLSX.readtable("data_ATIS3_new.xlsx", "s_prices")...)
for h in 1:24
    data["q_scenarios", h] = DataFrame(XLSX.readtable("data_ATIS3_new.xlsx", "s_q_$h")...)
end;

# Define Params

### Sets

In [4]:
# Number of scenarios, devided into FC scenarios and price scenarios
Ω_p = length(data["p_scenarios"].delta_15_da)
Ω_FC = 2*length(data["q_scenarios", 1].neg)
Ω = Ω_FC * Ω_p^2
# Number time periods
T = 1
# Number of hours per day
H = 24
# Number of FC levels (low, mid, high)
L = 3;

### Scalars

In [5]:
# Initial and final period
ti = 300
tf = ti + T-1
# Maximum deviation of DA bid from FC
FC_max_devDA = 0.15
# Risk
β = 0.0
α = 0.95
# Scaling factor FC
q_FC_scal = 0.025
# Scenario probablilities
pi = 1/Ω

2.5e-5

### Vectors

In [6]:
### Length T
# Central prices in DA
p_DA  = data["time_series"][ti:tf,"p_da"]
# 15 and DA prices are not used in the stochastic approach; p_DA * Δp_ instead
#p_15  = data["time_series"][ti:tf,"p_15"]
#p_ID = data["time_series"][ti:tf,"p_id3"]
# Power ForeCasted
q_FC = data["time_series"].fc_wind_bw[ti:tf] .* q_FC_scal    
# Forecast specifications; level ϵ [low,mid,high]; hour ϵ [1:24]
FC_level = fill(2, T)
FC_hour = []
FC_day = []
hour = 0
day = 1
for t in 1:T
    hour += 1
    if hour > 24
        hour = 1
        day +=1
    end
    push!(FC_hour, hour)
    push!(FC_day, day)
    if q_FC[t] <= 250*q_FC_scal
        FC_level[t] = 1
    elseif q_FC[t] > 1250*q_FC_scal
        FC_level[t] = 3
    end
end

### Length Ω_p
# Errors 15 and ID prices
Δp_15 = data["p_scenarios"].delta_15_da
Δp_ID = data["p_scenarios"].delta_id3_da

### Length Ω_FC
# FC error sign; 1st stage decission
FC_sgn = vcat(
    fill(1, length(data["q_scenarios", 1].neg)),
    fill(2, length(data["q_scenarios", 1].pos)));

**Explanation FC_type** \
The value of a forecast q_FC, as well as the hour of the day (time lag between FC and realization)
leads to different distributions of the FC error Δq_FC. We distinguish between the levels 1 (low; <250 MW), 2 (mid; 250 MW < Δq_FC < 1250 MW)
and 3 (high; > 1250 MW) for the forecasted value and devide the day in 24h. Depending on the period t, we save the hour and the day. Note that our hour has no connection to the "real" time. Later on, FC_level[t] and FC_hour[t] is used to select the right column of the forecast error distribution matrix Δq_FC. Concrete: Δq_FC[ω_FC, FC_level[t], FC_hour[t]].

### Matrices

In [7]:
# Forecast error matrix
    # Relative error of the forecast used for each scenario ω ϵ Ω,
    # depending on the forecast level F_level being low, mid or high (see text above)
    # and the hour of the day h
Δq_FC = Array{Float64,3}(undef, Ω_FC, L, H)
for h in 1:H
    Δq_FC[:,:,h] = cat(
        cat(data["q_scenarios", h].neg_u250, data["q_scenarios", h].pos_u250, dims=1),
        cat(data["q_scenarios", h].neg, data["q_scenarios", h].pos, dims=1),
        cat(data["q_scenarios", h].neg_o1250, data["q_scenarios", h].pos_o1250, dims=1),
        dims=2)
end
;

# Model - step by step

In [8]:
wind = Model(CPLEX.Optimizer);

In [9]:
# Power
q_DA = @variable(wind, [1:T]) # No ω dependence due to non-anticipativity
q_15 = @variable(wind, [1:T, 1:2]) # Only two decissions, depending on the sign of the FC
# q_ID is in fact not needed, as it only depends on q_DA, q_15 and q_r=q_FC[t]*(1+Δq_FC[ω])

# Risk
η = @variable(wind, [1:T, 1:Ω_FC, 1:Ω_p, 1:Ω_p])
ζ = @variable(wind, [1:T]);

In [10]:
# Realized power
q_r = @expression(wind, [t in 1:T, ω_FC in 1:Ω_FC],
    q_FC[t] * (1 + Δq_FC[ω_FC,FC_level[t],FC_hour[t]]))
# Realized prices
p_15 = @expression(wind, [t in 1:T, ω_15 in 1:Ω_p],
    p_DA[t] * (1 + Δp_15[ω_15]))
p_ID = @expression(wind, [t in 1:T, ω_ID in 1:Ω_p],
    p_DA[t] * (1 + Δp_ID[ω_ID]))
# Revenue
R = @expression(wind, [t in 1:T, ω_FC in 1:Ω_FC, ω_15 in 1:Ω_p, ω_ID in 1:Ω_p],
    p_DA[t] * q_DA[t]
    + p_15[t,ω_15] * q_15[t,FC_sgn[ω_FC]]
    + p_ID[t,ω_ID] * (q_r[t,ω_FC] - q_DA[t] - q_15[t,FC_sgn[ω_FC]]))
# Risk
CVaR = @expression(wind, [t in 1:T],
    (ζ[t] - 1/(1-α) * sum(sum(sum(
                pi*η[t,ω_FC,ω_15,ω_ID]
            for ω_ID in 1:Ω_p)
        for ω_15 in 1:Ω_p)
    for ω_FC in 1:Ω_FC)
    )
);

In [11]:
@objective(wind, Max,
    sum(sum(sum(sum(
                    pi*R[t,ω_FC,ω_15,ω_ID] 
                for ω_ID in 1:Ω_p)
            for ω_15 in 1:Ω_p)
        for ω_FC in 1:Ω_FC)
        + β * CVaR[t]
    for t in 1:T)
);

In [12]:
# DA bid within certain percentage of the FC
MinMaxPowerDA = @constraint(wind, [t in 1:T],
    (1-FC_max_devDA) * q_FC[t] <= q_DA[t] <= (1+FC_max_devDA) * q_FC[t]);
# Limits for total sold power; max is given by realization of last scenario of each period
MaxPowerTot = @constraint(wind, [t in 1:T, ω_FC in 1:Ω_FC],
    0 <= q_DA[t]+q_15[t,FC_sgn[ω_FC]] <= q_r[t,end]); 
# Risk
CVaRConst = @constraint(wind, [t in 1:T, ω_FC in 1:Ω_FC, ω_15 in 1:Ω_p, ω_ID in 1:Ω_p],
    ζ[t] - R[t,ω_FC,ω_15,ω_ID] <= η[t,ω_FC,ω_15,ω_ID])
ηLimLow = @constraint(wind, [t in 1:T, ω_FC in 1:Ω_FC, ω_15 in 1:Ω_p, ω_ID in 1:Ω_p],
    η[t,ω_FC,ω_15,ω_ID] >= 0);

In [13]:
optimize!(wind)
termination_status(wind)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 80000 rows and 40001 columns.
Aggregator did 1 substitutions.
Reduced LP has 100 rows, 103 columns, and 300 nonzeros.
Presolve time = 0.05 sec. (31.28 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =           483.814696

Dual simplex solved model.



OPTIMAL::TerminationStatusCode = 1

# Model as Function

In [14]:
function wind_opt(β=0.0, FC_max_devDA=0.15, t_i=300, T=1)
    wind = Model(CPLEX.Optimizer)
    set_silent(wind)

    @variables(wind, begin
        # Powers
        q_DA[1:T] >= 0
        q_15[1:T,1:2]
        # Risk
        η[1:T, 1:Ω_FC, 1:Ω_p, 1:Ω_p] >= 0
        ζ[1:T]
    end)    
    
    q_r = @expression(wind, [t in 1:T, ω_FC in 1:Ω_FC], # Realized power
        q_FC[t] * (1 + Δq_FC[ω_FC,FC_level[t],FC_hour[t]]))
    p_15 = @expression(wind, [t in 1:T, ω_15 in 1:Ω_p], # Realized 15 price
        p_DA[t] * (1 + Δp_15[ω_15]))
    p_ID = @expression(wind, [t in 1:T, ω_ID in 1:Ω_p], # Realized ID price
        p_DA[t] * (1 + Δp_ID[ω_ID]))
    R = @expression(wind, [t in 1:T, ω_FC in 1:Ω_FC, ω_15 in 1:Ω_p, ω_ID in 1:Ω_p], # Revenue
        p_DA[t] * q_DA[t]
        + p_15[t,ω_15] * q_15[t,FC_sgn[ω_FC]]
        + p_ID[t,ω_ID] * (q_r[t,ω_FC] - q_DA[t] - q_15[t,FC_sgn[ω_FC]]))
    CVaR = @expression(wind, [t in 1:T], # Risk
        (ζ[t] - 1/(1-α) * sum(sum(sum(
                    pi*η[t,ω_FC,ω_15,ω_ID]
                for ω_ID in 1:Ω_p)
            for ω_15 in 1:Ω_p)
        for ω_FC in 1:Ω_FC)));
    
    @objective(wind, Max,
        sum(sum(sum(sum(
                        pi*R[t,ω_FC,ω_15,ω_ID] 
                    for ω_ID in 1:Ω_p)
                for ω_15 in 1:Ω_p)
            for ω_FC in 1:Ω_FC)
            + β * CVaR[t]
        for t in 1:T));

    @constraints(wind, begin
        # DA bid within certain percentage of the FC
        MinMaxPowerDA[t in 1:T], (1-FC_max_devDA) * q_FC[t] <= q_DA[t] <= (1+FC_max_devDA) * q_FC[t]
        # Limits for total sold power
        MaxPowerTot[t in 1:T, ω_FC in 1:Ω_FC], 0 <= q_DA[t]+q_15[t,FC_sgn[ω_FC]] <= q_r[t,end]
        # Risk
        CVaRconstr[t in 1:T, ω_FC in 1:Ω_FC, ω_15 in 1:Ω_p, ω_ID in 1:Ω_p],
            ζ[t] - R[t,ω_FC,ω_15,ω_ID] <= η[t,ω_FC,ω_15,ω_ID]
    end)    
    
    optimize!(wind)
    
    result = DataFrame(
        β = Float64[],
        p_DA = Float64[],
        p_15 = Float64[],
        p_ID = Float64[],
        q_FC = Float64[],
        q_DA = Float64[],
        q_15_neg = Float64[],
        q_15_pos = Float64[],
        R_mean = Float64[],
        R_STD = Float64[],
        CVaR = Float64[],
    )
    for t in 1:T
        push!(result, [
            β, p_DA[t], p_15[t], p_ID[t], q_FC[t],
            value.(q_DA[t]),
            value.(q_15[t,1]),
            value.(q_15[t,2]),
            Statistics.mean(value.(R[t,:,:,:])),
            Statistics.std(value.(R[t,:,:,:])),
            value.(CVaR[t])
            ])
    end
    return result
end;

In [15]:
result = wind_opt()
for β in 0.2:0.2:2
    push!(result, wind_opt(β)[1,:])
end

In [16]:
show(result, allcols=true)

11×11 DataFrame
 Row │ β        p_DA     p_15     p_ID      q_FC     q_DA     q_15_neg  q_15_pos  R_mean   R_STD    CVaR     
     │ Float64  Float64  Float64  Float64   Float64  Float64  Float64   Float64   Float64  Float64  Float64  
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │     0.0    33.56  14.9117  -8.51317  14.1125  16.2294   8.17523   8.17523  482.14   168.726  -13.3699
   2 │     0.2    33.56  14.9117  -8.51317  14.1125  16.2294  -4.38319   7.00063  476.722  128.34   209.735
   3 │     0.4    33.56  14.9117  -8.51317  14.1125  16.2294  -5.48994   6.15599  475.953  125.838  212.692
   4 │     0.6    33.56  14.9117  -8.51317  14.1125  16.031   -5.42104   5.89513  475.635  124.225  213.3
   5 │     0.8    33.56  14.9117  -8.51317  14.1125  14.1096  -3.98167   6.65264  474.154  121.093  215.456
   6 │     1.0    33.56  14.9117  -8.51317  14.1125  13.0536  -3.16434   6.73657  473.22   118.802  216.51
   7 │  